# 前処理
- データ解析の８割は前処理
    - 欠損値への対応
    - カテゴリ変数のエンコーディング
    - 特徴量の正規化
- pandasとの違い
    - scikit-learnでは、クラスをインスタンス化してfitメソッドとtransformメソッドを用いて前処理が可能
    - scikit-learnでは、全体で統一的なインターフェースでわかりやすい

## 欠損値への対応
1. 除去
1. 補完

In [1]:
import numpy as np
import pandas as pd

# サンプルデータを作成
df = pd.DataFrame(
    {
        'A':[1,np.nan, 3, 4, 5],
        'B':[6, 7, 8, np.nan, 10],
        'C':[11, 12, 13, 14, 15]
    }
)

df

,A,B,C
0,1.0,6.0,11
1,NaN,7.0,12
2,3.0,8.0,13
3,4.0,NaN,14
4,5.0,10.0,15


### 欠損値の確認(isnullメソッド)

In [11]:
# 欠損値をを確認する
df.isnull()

,A,B,C
0,False,False,False
1,True,False,False
2,False,False,False
3,False,True,False
4,False,False,False


### 欠損値の除去(dropnaメソッド,fillnaメソッド)

In [12]:
# 欠損値がある行を削除
df.dropna()

,A,B,C
0,1.0,6.0,11
2,3.0,8.0,13
4,5.0,10.0,15


In [13]:
# 欠損値を0で埋める
df.fillna(0)

,A,B,C
0,1.0,6.0,11
1,0.0,7.0,12
2,3.0,8.0,13
3,4.0,0.0,14
4,5.0,10.0,15


In [14]:
# 欠損値を１つ手前の値で補完する
df.fillna(method='ffill')

,A,B,C
0,1.0,6.0,11
1,1.0,7.0,12
2,3.0,8.0,13
3,4.0,8.0,14
4,5.0,10.0,15


### 欠損値の補完(SimpleImputer)

In [15]:
# 欠損値の補完
# 列ごとに平均を計算して補完する

from sklearn.impute import SimpleImputer

# 平均値で欠損値を補完するインスタンスを作成
imp = SimpleImputer(strategy='mean') # mean,median,most_frequentを選択できる

# 欠損値を補完
imp.fit(df)
imp.transform(df)

array([[ 1.  ,  6.  , 11.  ],
       [ 3.25,  7.  , 12.  ],
       [ 3.  ,  8.  , 13.  ],
       [ 4.  ,  7.75, 14.  ],
       [ 5.  , 10.  , 15.  ]])

## カテゴリ変数のエンコーディング
- 2つの方法がある
    - カテゴリ変数のエンコーディング（a→0,b→1,c→2...）
    - One-hotエンコーディング

In [21]:
import pandas as pd

df = pd.DataFrame(
    {
        'A':[1,2,3,4,5],
        'B':['a','b','a','b','c']
    }
)

df

,A,B
0,1,a
1,2,b
2,3,a
3,4,b
4,5,c


### エンコーディング（LabelEncoder）

In [28]:
from sklearn.preprocessing import LabelEncoder

# ラベルエンコーダーのインスタンスを生成
le = LabelEncoder()

# ラベルのエンコーディング
le.fit(df['B'])
le.transform(df['B'])

array([0, 1, 0, 1, 2])

In [29]:
# 返還された値と元の値の対応は、LabelEncoderのインスタンスのclasses_属性で確認できる

le.classes_

array(['a', 'b', 'c'], dtype=object)

### One-hotエンコーディング(sklearnでやる場合)

In [38]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# DataFrameをコピー
df_ohe = df.copy()

# ラベルエンコーダーのインスタンスを生成
le = LabelEncoder()

# ラベルのエンコーディング
le.fit(df_ohe['B'])
df_ohe['B'] = le.transform(df_ohe['B'])

# One-hotエンコーダのインスタンス化
ohe = ColumnTransformer([("OneHotEncoder", OneHotEncoder(), [1])],remainder='passthrough')

ohe.fit_transform(df_ohe)

array([[1., 0., 0., 1.],
       [0., 1., 0., 2.],
       [1., 0., 0., 3.],
       [0., 1., 0., 4.],
       [0., 0., 1., 5.]])

### One-hotエンコーディング（pd.get_dummiesのほうが一発でできる）

In [40]:
pd.get_dummies(df)

,A,B_a,B_b,B_c
0,1,1,0,0
1,2,0,1,0
2,3,1,0,0
3,4,0,1,0
4,5,0,0,1


## 特徴量の正規化